

### Importing Libraries and Dataset


In [ ]:
import numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
import zipfile
!pip install xgboost


In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler , StandardScaler , PowerTransformer
from sklearn.model_selection import train_test_split ,  KFold , cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB , BernoulliNB
from sklearn.metrics import confusion_matrix , accuracy_score , precision_score, recall_score, f1_score , ConfusionMatrixDisplay

In [ ]:
!pip install kaggle
from google.colab import files
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle
!kaggle datasets download -d shayanfazeli/heartbeat
!unzip heartbeat.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/shayanfazeli/heartbeat
License(s): unknown
 97% 96.0M/98.8M [00:00<00:00, 224MB/s]
100% 98.8M/98.8M [00:00<00:00, 199MB/s]
Archive:  heartbeat.zip
  inflating: mitbih_test.csv         
  inflating: mitbih_train.csv        
  inflating: ptbdb_abnormal.csv      
  inflating: ptbdb_normal.csv        


### No sampling log reg


In [ ]:
import pandas as pd
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

train = pd.read_csv('mitbih_train.csv', header=None)
test = pd.read_csv('mitbih_test.csv', header=None)

x_train = train.iloc[:, :100]
y_train = train.iloc[:, 187]
x_test = test.iloc[:, :100]
y_test = test.iloc[:, 187]

print(f"Original dataset shape: {Counter(y_train)}")

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

log_reg = LogisticRegression(max_iter=1000, class_weight='balanced')

log_reg.fit(x_train, y_train)

y_pred = log_reg.predict(x_test)

print("Classification Report for Logistic Regression:")
print(classification_report(y_test, y_pred))


Original dataset shape: Counter({0.0: 72471, 4.0: 6431, 2.0: 5788, 1.0: 2223, 3.0: 641})
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

         0.0       0.97      0.63      0.77     18118
         1.0       0.14      0.65      0.23       556
         2.0       0.27      0.75      0.40      1448
         3.0       0.09      0.86      0.16       162
         4.0       0.76      0.90      0.82      1608

    accuracy                           0.66     21892
   macro avg       0.45      0.76      0.48     21892
weighted avg       0.88      0.66      0.73     21892



### ***Binary classifier log reg***

In [ ]:
import pandas as pd
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('mitbih_train.csv', header=None)
test = pd.read_csv('mitbih_test.csv', header=None)

x = train.iloc[:, :100]
y = train.iloc[:, 187]

y_binary = (y == 0).astype(int)

x_test = test.iloc[:, :100]
y_test = test.iloc[:, 187]

y_test_binary = (y_test == 0).astype(int)

print(f"Original dataset shape: {Counter(y_binary)}")

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_test_scaled = scaler.transform(x_test)

log_reg = LogisticRegression(max_iter=1000, class_weight='balanced')
log_reg.fit(x_scaled, y_binary)

y_pred = log_reg.predict(x_test_scaled)

print("Classification Report for Logistic Regression (Binary Classification):")
print(classification_report(y_test_binary, y_pred))


Original dataset shape: Counter({1: 72471, 0: 15083})
Classification Report for Logistic Regression (Binary Classification):
              precision    recall  f1-score   support

           0       0.53      0.76      0.62      3774
           1       0.94      0.86      0.90     18118

    accuracy                           0.84     21892
   macro avg       0.73      0.81      0.76     21892
weighted avg       0.87      0.84      0.85     21892



### Random Forrest 5 cat

In [ ]:
import pandas as pd
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('mitbih_train.csv', header=None)
test = pd.read_csv('mitbih_test.csv', header=None)

x_train = train.iloc[:, :100]
y_train = train.iloc[:, 187]
x_test = test.iloc[:, :100]
y_test = test.iloc[:, 187]

print(f"Original dataset shape: {Counter(y_train)}")

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

rf_model = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)

rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_test)

print("Classification Report for Random Forest:")
print(classification_report(y_test, y_pred))


Original dataset shape: Counter({0.0: 72471, 4.0: 6431, 2.0: 5788, 1.0: 2223, 3.0: 641})
Classification Report for Random Forest:
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99     18118
         1.0       0.96      0.59      0.73       556
         2.0       0.98      0.88      0.93      1448
         3.0       0.83      0.59      0.69       162
         4.0       0.99      0.95      0.97      1608

    accuracy                           0.97     21892
   macro avg       0.95      0.80      0.86     21892
weighted avg       0.97      0.97      0.97     21892



### 2 models Random Forrest

In [ ]:
# Phase 1
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

train = pd.read_csv('mitbih_train.csv', header=None)
test = pd.read_csv('mitbih_test.csv', header=None)

X_train, y_train = train.iloc[:, :100], train.iloc[:, 187]
X_test, y_test = test.iloc[:, :100], test.iloc[:, 187]

y_train_binary = (y_train == 0).astype(int)
y_test_binary = (y_test == 0).astype(int)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train_binary)

print("First Model (Binary Classification):")
y_pred_binary = rf_model.predict(X_test)
print(classification_report(y_test_binary, y_pred_binary))
print(f"ROC-AUC Score: {roc_auc_score(y_test_binary, y_pred_binary):.2f}")

y_train_pred_binary = rf_model.predict(X_train)
y_test_pred_binary = rf_model.predict(X_test)


First Model (Binary Classification):
              precision    recall  f1-score   support

           0       0.99      0.88      0.93      3774
           1       0.98      1.00      0.99     18118

    accuracy                           0.98     21892
   macro avg       0.98      0.94      0.96     21892
weighted avg       0.98      0.98      0.98     21892

ROC-AUC Score: 0.94


In [ ]:
# Phase 2 no over sample
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

train = pd.read_csv('mitbih_train.csv', header=None)
test = pd.read_csv('mitbih_test.csv', header=None)

X_train, y_train = train.iloc[:, :100], train.iloc[:, 187]
X_test, y_test = test.iloc[:, :100], test.iloc[:, 187]

print("Class distribution in training set:")
print(y_train.value_counts())

X_train_subclass = X_train[y_train.isin([1, 2, 3, 4])]
y_train_subclass = y_train[y_train.isin([1, 2, 3, 4])]

X_test_subclass = X_test[y_test.isin([1, 2, 3, 4])]
y_test_subclass = y_test[y_test.isin([1, 2, 3, 4])]

print(f"Number of subclass samples in training set: {y_train_subclass.shape[0]}")
print(f"Number of subclass samples in test set: {y_test_subclass.shape[0]}")

if y_train_subclass.shape[0] > 0:
    class_weights = {
        1: 2.0,
        2: 2.0,
        3: 20.0,
        4: 2.0
    }

    rf_model_subclass = RandomForestClassifier(

        class_weight=class_weights,
        random_state=42
    )

    rf_model_subclass.fit(X_train_subclass, y_train_subclass)

    print("Second Model (Multi-Class Classification with Class Weights):")
    y_pred_subclass = rf_model_subclass.predict(X_test_subclass)
    print(classification_report(y_test_subclass, y_pred_subclass))
else:
    print("No samples available for training the subclass model.")


Class distribution in training set:
187
0.0    72471
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: count, dtype: int64
Number of subclass samples in training set: 15083
Number of subclass samples in test set: 3774
Second Model (Multi-Class Classification with Class Weights):
              precision    recall  f1-score   support

         1.0       0.97      0.93      0.95       556
         2.0       0.95      0.98      0.96      1448
         3.0       0.85      0.80      0.83       162
         4.0       0.99      0.98      0.99      1608

    accuracy                           0.96      3774
   macro avg       0.94      0.92      0.93      3774
weighted avg       0.96      0.96      0.96      3774



Over Sample

In [ ]:
# Phase 2

!pip install imbalanced-learn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

train = pd.read_csv('mitbih_train.csv', header=None)
test = pd.read_csv('mitbih_test.csv', header=None)

X_train, y_train = train.iloc[:, :100], train.iloc[:, 187]
X_test, y_test = test.iloc[:, :100], test.iloc[:, 187]

print("Class distribution in training set:")
print(y_train.value_counts())

X_train_subclass = X_train[y_train.isin([1, 2, 3, 4])]
y_train_subclass = y_train[y_train.isin([1, 2, 3, 4])]

X_test_subclass = X_test[y_test.isin([1, 2, 3, 4])]
y_test_subclass = y_test[y_test.isin([1, 2, 3, 4])]

print(f"Number of subclass samples in training set: {y_train_subclass.shape[0]}")
print(f"Number of subclass samples in test set: {y_test_subclass.shape[0]}")

smote = SMOTE(sampling_strategy={3: 7000}, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_subclass, y_train_subclass)

class_weights = {
        1: 2.0,
        2: 2.0,
        3: 20.0,
        4: 2.0
    }

rf_model_subclass = RandomForestClassifier(
        max_depth=None,
        class_weight=class_weights,
        random_state=42
    )

rf_model_subclass.fit(X_resampled, y_resampled)

print("Second Model (Multi-Class Classification with Class Weights):")
    y_pred_subclass = rf_model_subclass.predict(X_test_subclass)
print(classification_report(y_test_subclass, y_pred_subclass))


Class distribution in training set:
187
0.0    72471
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: count, dtype: int64
Number of subclass samples in training set: 15083
Number of subclass samples in test set: 3774
Second Model (Multi-Class Classification with Class Weights):
              precision    recall  f1-score   support

         1.0       0.97      0.92      0.94       556
         2.0       0.95      0.97      0.96      1448
         3.0       0.82      0.88      0.85       162
         4.0       0.99      0.98      0.99      1608

    accuracy                           0.96      3774
   macro avg       0.93      0.94      0.93      3774
weighted avg       0.96      0.96      0.96      3774



### 2 Models  Xgboost  (Best One)

---





In [20]:
#phase 1
!pip install imbalanced-learn
!pip install xgboost

import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

train = pd.read_csv('mitbih_train.csv', header=None)
test = pd.read_csv('mitbih_test.csv', header=None)

X_train, y_train = train.iloc[:, :100], train.iloc[:, 187]
X_test, y_test = test.iloc[:, :100], test.iloc[:, 187]

y_train_binary = (y_train != 0).astype(int)
y_test_binary = (y_test != 0).astype(int)

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train_binary)

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_resampled, y_resampled)

y_pred_binary = xgb_model.predict(X_test)

print("XGBoost Model (Binary Classification):")
print(classification_report(y_test_binary, y_pred_binary))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:12:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Model (Binary Classification):
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18118
           1       0.93      0.93      0.93      3774

    accuracy                           0.98     21892
   macro avg       0.96      0.96      0.96     21892
weighted avg       0.98      0.98      0.98     21892



In [26]:
#Phase 2
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter

train = pd.read_csv('mitbih_train.csv', header=None)
test = pd.read_csv('mitbih_test.csv', header=None)

X_train, y_train = train.iloc[:, :100], train.iloc[:, 187]
X_test, y_test = test.iloc[:, :100], test.iloc[:, 187]

X_train_subclass = X_train[y_train.isin([1, 2, 3, 4])]
y_train_subclass = y_train[y_train.isin([1, 2, 3, 4])] - 1
X_test_subclass = X_test[y_test.isin([1, 2, 3, 4])]
y_test_subclass = y_test[y_test.isin([1, 2, 3, 4])]  - 1

print("Class distribution in training set:")
print(y_train.value_counts())

print(f"Number of subclass samples in training set: {y_train_subclass.shape[0]}")
print(f"Number of subclass samples in test set: {y_test_subclass.shape[0]}")

smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_subclass, y_train_subclass)
print("Resampled training data distribution:", Counter(y_resampled))



Abnormal_model = XGBClassifier(objective='multi:softmax',
                          num_class=4,
                          random_state=42,
                          eval_metric='logloss',
                          n_estimators = 1000 ,
                          learning_rate = .1 ,
                          subsample =.8 ,
                          colsample_bytree = .8 ,
                          max_depth = 6)


Abnormal_model.fit(X_resampled, y_resampled)

y_pred_subclass = Abnormal_model.predict(X_test_subclass) + 1


print("Model Performance (XGBoost with SMOTE - Custom Strategy):")
print(classification_report(y_test_subclass + 1, y_pred_subclass))


Class distribution in training set:
187
0.0    72471
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: count, dtype: int64
Number of subclass samples in training set: 15083
Number of subclass samples in test set: 3774
Resampled training data distribution: Counter({0.0: 6431, 1.0: 6431, 2.0: 6431, 3.0: 6431})
Model Performance (XGBoost with SMOTE - Custom Strategy):
              precision    recall  f1-score   support

         1.0       0.98      0.95      0.96       556
         2.0       0.97      0.97      0.97      1448
         3.0       0.81      0.91      0.85       162
         4.0       0.99      0.99      0.99      1608

    accuracy                           0.97      3774
   macro avg       0.94      0.95      0.94      3774
weighted avg       0.97      0.97      0.97      3774



### Saving Models

In [28]:
import pickle

with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

with open('Abnormal_model.pkl', 'wb') as f:
    pickle.dump(Abnormal_model, f)




In [29]:
from google.colab import files
files.download('Abnormal_model.pkl')
files.download('xgb_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Github === https://github.com/Shady-Abdelaziz/ECG-Classifications-Using-2-Models/tree/main
 App.py === https://github.com/Shady-Abdelaziz/ECG-Classifications-Using-2-Models/blob/main/App.py